In [1]:
import tensorflow as tf
import pandas as pd

Here, TensorFlow is imported for creating and training the neural network model, 
and pandas is imported for data manipulation.

In [2]:
csv_file_path = 'game_states.csv'
data = pd.read_csv(csv_file_path)

The game state data is read from a CSV file into a pandas DataFrame.

In [3]:
data['key_up'].replace({'a': 0, 'd': 1}, inplace=True)
data['key_up'].fillna(2, inplace=True)
data['key_down'].replace({'LEFT': 0, 'RIGHT': 1}, inplace=True)
data['key_down'].fillna(2, inplace=True)

Here, the key press data for the 'up' paddle and 'down' paddle are converted into numerical values: 
- 'a' or 'LEFT' is converted to 0, 
- 'd' or 'RIGHT' to 1, 
- and NaN (no key press) to 2.

In [4]:
X = data.drop(columns=['point', 'key_up', 'key_down'])
y = data['key_up']

The features X and labels y for the model are defined. 
The features include all columns except 'point', 'key_up', and 'key_down'. 
The labels are set to be the 'key_up' column, which we're trying to predict.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

The data is split into training and testing sets using a 80-20 split.
- X and y: These are your features and labels, respectively.
- test_size=0.2: This specifies that 20% of the data will be used for testing, and the remaining 80% will be used for training.
- random_state=42: This is a random seed for reproducibility. 
Using the same random_state will produce the same split each time the code is run, which is useful for debugging and comparison.

In [7]:
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

A function to create the neural network model is defined. 
- The model has an input layer, followed by three hidden layers with 128, 64, and 32 neurons, 
and an output layer with 3 neurons (corresponding to the three possible actions).
- The InputLayer is initialized with the shape of the data it will receive. This helps TensorFlow optimize the computations.
- The first layer is a densely-connected neural network layer with 128 neurons and a ReLU (Rectified Linear Unit) activation function.
- Finally, the output layer has 3 neurons, each representing one of the possible actions ('Left', 'Right', 'Stay'). The 'softmax' activation function is used to turn the output into a probability distribution.
- Then, you specify the optimizer, loss function, and metrics you want to track. The Adam optimizer and sparse categorical cross-entropy loss are commonly used for classification problems.

In [8]:
input_shape = X_train.shape[1]
model = create_model(input_shape)
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
16/16 [==============================] - 0s 6ms/step - loss: 12.4760 - accuracy: 0.8169 - val_loss: 5.8603 - val_accuracy: 0.9344
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 5.4364 - accuracy: 0.8992 - val_loss: 3.0479 - val_accuracy: 0.8607
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 2.3977 - accuracy: 0.8889 - val_loss: 1.3444 - val_accuracy: 0.8525
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6507 - accuracy: 0.9074 - val_loss: 0.4957 - val_accuracy: 0.9098
Epoch 5/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4428 - accuracy: 0.9095 - val_loss: 0.5188 - val_accuracy: 0.9344
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.9239 - val_loss: 0.3032 - val_accuracy: 0.9262
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.3562 - accuracy: 0.9239 - val_loss: 0.3879 - val_accuracy: 0.9344
Epoch 8/100


The model is trained using the training data for 100 epochs. The batch size is set to 32.

In [9]:
model.save('pong_game_model.h5')

/Users/enricomelis/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Finally, the trained model is saved to a file named pong_game_model.h5.